In [15]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.regularizers import l2

Attribute Information
Input features in order:
1) CRIM: per capita crime rate by town
2) ZN: proportion of residential land zoned for lots over 25,000 sq.ft.
3) INDUS: proportion of non-retail business acres per town
4) CHAS: Charles River dummy variable (1 if tract bounds river; 0 otherwise)
5) NOX: nitric oxides concentration (parts per 10 million) [parts/10M]
6) RM: average number of rooms per dwelling
7) AGE: proportion of owner-occupied units built prior to 1940
8) DIS: weighted distances to five Boston employment centres
9) RAD: index of accessibility to radial highways
10) TAX: full-value property-tax rate per $10,000 [$/10k]
11) PTRATIO: pupil-teacher ratio by town
12) B: The result of the equation B=1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
13) LSTAT: % lower status of the population

Output variable:
1) MEDV: Median value of owner-occupied homes in $1000's [k$]

In [16]:
TRAIN_TEST_SPLIT = 0.8

Loading the data

In [17]:
col_names = ["CRIM", "ZN", "INDUS", "CHAS", "NOX","RM", "AGE", "DIS","RAD", "TAX", "PTRATIO", "B", "LSTAT","MEDV"]
data = pd.read_csv('housing.csv', sep="\s+", names=col_names)

Checking for nulls

In [18]:
data.isnull().values.any()

False

In [19]:
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


Normalization and preprocessing

In [20]:
input_data = data[data.columns[:13]]
output_data = data[data.columns[13]]

scaler = MinMaxScaler()

input_data = pd.DataFrame(scaler.fit_transform(input_data), columns=data.columns[:13])


In [21]:
data = pd.concat([input_data,output_data], axis=1)
data = data.to_numpy()
np.random.shuffle(data)
input_data = data[:,:13]
output_data = data[:,13]

In [22]:
X_train = input_data[:int(TRAIN_TEST_SPLIT*input_data.shape[0])]
y_train = output_data[:int(TRAIN_TEST_SPLIT*input_data.shape[0])]
X_test = input_data[int(TRAIN_TEST_SPLIT*input_data.shape[0]):]
y_test = output_data[int(TRAIN_TEST_SPLIT*input_data.shape[0]):]

Creating a Dense Model, fitting and evaluation

In [23]:
model = tf.keras.models.Sequential(
    [
        layers.Dense(13, activation='relu'),
        layers.Dense(512, activation='relu',kernel_regularizer=l2(0.01)),
        layers.Dense(256,activation='relu', kernel_regularizer=l2(0.01)),
        layers.Dense(32,activation='relu', kernel_regularizer=l2(0.01)),
        layers.Dense(1, activation='linear')
    ]
)

In [24]:
model.compile(
    loss=tf.keras.losses.MSE,
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
)

In [25]:
model.fit(X_train, y_train, epochs=100, batch_size=16, verbose=2)

Epoch 1/100
26/26 - 1s - loss: 492.2241 - 593ms/epoch - 23ms/step
Epoch 2/100
26/26 - 0s - loss: 164.1767 - 32ms/epoch - 1ms/step
Epoch 3/100
26/26 - 0s - loss: 102.9739 - 31ms/epoch - 1ms/step
Epoch 4/100
26/26 - 0s - loss: 76.4155 - 39ms/epoch - 1ms/step
Epoch 5/100
26/26 - 0s - loss: 65.0315 - 31ms/epoch - 1ms/step
Epoch 6/100
26/26 - 0s - loss: 56.8803 - 39ms/epoch - 2ms/step
Epoch 7/100
26/26 - 0s - loss: 47.0570 - 32ms/epoch - 1ms/step
Epoch 8/100
26/26 - 0s - loss: 45.3853 - 32ms/epoch - 1ms/step
Epoch 9/100
26/26 - 0s - loss: 38.2611 - 31ms/epoch - 1ms/step
Epoch 10/100
26/26 - 0s - loss: 35.2815 - 29ms/epoch - 1ms/step
Epoch 11/100
26/26 - 0s - loss: 31.7600 - 32ms/epoch - 1ms/step
Epoch 12/100
26/26 - 0s - loss: 30.8552 - 28ms/epoch - 1ms/step
Epoch 13/100
26/26 - 0s - loss: 30.8339 - 27ms/epoch - 1ms/step
Epoch 14/100
26/26 - 0s - loss: 29.0239 - 29ms/epoch - 1ms/step
Epoch 15/100
26/26 - 0s - loss: 27.3328 - 30ms/epoch - 1ms/step
Epoch 16/100
26/26 - 0s - loss: 27.6212 - 33

In [26]:
model.evaluate(X_test, y_test, verbose=2)

4/4 - 0s - loss: 12.0377 - 113ms/epoch - 28ms/step


12.037691116333008

Not too well, but we also didn't have much data

In [27]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 13)                182       
                                                                 
 dense_6 (Dense)             (None, 512)               7168      
                                                                 
 dense_7 (Dense)             (None, 256)               131328    
                                                                 
 dense_8 (Dense)             (None, 32)                8224      
                                                                 
 dense_9 (Dense)             (None, 1)                 33        
                                                                 
Total params: 146,935
Trainable params: 146,935
Non-trainable params: 0
_________________________________________________________________


Data from: https://www.kaggle.com/datasets/vikrishnan/boston-house-prices